## LION pile

pile object is used throughout LION system, it's an async-compatible strong-typed ordered dict contains only instances of subclasses of `lionabc.Observable`

`Observable` objects has `ln_id`


In [1]:
import lionfuncs as ln
from lion_core.generic.component import Component
from lion_core.generic.element import Element
from lion_core.generic.pile import Pile

## Setitem / getitem

The heart of LION system data structure philosophy

In [2]:
from pydantic import Field


# Example Element class for demonstration
class Book(Element):
    title: str = Field(..., title="The title of the book")
    author: str = Field(..., title="The author of the book")

    def __init__(self, title: str, author: str):
        super().__init__(title=title, author=author)

    def __repr__(self):
        return f"Book('{self.title}' by {self.author})"

In [3]:
books = Pile(
    item_type=Book, strict_type=True
)  # this means we can only add Book objects to the pile
books.to_dict()  # strict_type=True means, not even Book subclasses can be added

{'timestamp': 1727789378.053981,
 'ln_id': 'ln8fbd85-9ac5e4c842fab5-29f7a45da-5a003c-e24862b',
 'pile': [],
 'strict_type': False,
 'lion_class': 'Pile'}

In [4]:
# set item and the many ways of using it

# 1. set item by item
book1 = Book(title="The Hobbit", author="J.R.R. Tolkien")
books[book1] = book1
print(len(books))

1


In [5]:
# 2. set a list of items by int
_books = [
    Book(title="The Hobbit", author="J.R.R. Tolkien"),
    Book(title="The Lord of the Rings", author="J.R.R. Tolkien"),
]
books[0] = _books

print(len(books))

2


In [6]:
# 3. set a list of items by slice
_books = [
    Book(title="The Hobbit", author="J.R.R. Tolkien"),
    Book(title="The Lord of the Rings", author="J.R.R. Tolkien"),
    Book("1984", "George Orwell"),
]
books[1:] = _books

print(len(books))

4


In [7]:
# 4. set item by item id
book2 = Book(title="The Hobbit", author="J.R.R. Tolkien")
books[book2.ln_id] = book2
book2 in books

True

In [8]:
# 5. set a dict of item, by a dict of items
book_dict_ = {
    "1": Book(title="The Hobbit", author="J.R.R. Tolkien"),
    "2": Book(title="The Lord of the Rings", author="J.R.R. Tolkien"),
    "3": Book(title="1984", author="George Orwell"),
}

books[book_dict_] = book_dict_
book_dict_ in books

True

In [9]:
len(books)

8

There are more ways of using setitem/getitem, it supports many input data types (but must be of LION objects)

In [10]:
# get item and the equally many ways of using it, by int
books[0]
print(len(books))

8


In [11]:
books[1:3]  # by slice

[Book('The Hobbit' by J.R.R. Tolkien),
 Book('The Lord of the Rings' by J.R.R. Tolkien)]

In [12]:
books[book2.ln_id]  # by item ln_id

Book('The Hobbit' by J.R.R. Tolkien)

In [13]:
books[book2]  # by item, i'm not sure why you would want to do this, but it's possible

Book('The Hobbit' by J.R.R. Tolkien)

In [14]:
print(books.get(10, None))  # by index, with default value for error handling

None


## Include / Update / Insert / Append

In [15]:
# Initialize a Pile
library = Pile(item_type=Book)

In [16]:
# include: Add an item or items to the pile
library.include(Book("1984", "George Orwell"))
library.include(
    [
        Book("To Kill a Mockingbird", "Harper Lee"),
        Book("Pride and Prejudice", "Jane Austen"),
    ]
)
print(f"Library after include: {library}")

Library after include: Pile(3)


In [17]:
# update: Update the pile with items from another iterable
new_books = [
    Book("The Great Gatsby", "F. Scott Fitzgerald"),
    Book("One Hundred Years of Solitude", "Gabriel García Márquez"),
]
library.update(new_books)
print(f"Library after update: {library}")

Library after update: Pile(5)


In [18]:
# insert: Insert an item at a specific position
library.insert(2, Book("Moby-Dick", "Herman Melville"))
print(f"Library after insert: {library}")

Library after insert: Pile(6)


In [19]:
# append: Add an item to the end of the pile
library.append(Book("The Hobbit", "J.R.R. Tolkien"))
print(f"Library after append: {library}")

Library after append: Pile(7)


In [20]:
# Demonstrate error handling and type checking
try:
    library.include("Not a Book object")
except Exception as e:
    print(f"Error when trying to include non-Book object: {e}")

In [21]:
# Showcase the use of strict type checking
strict_library = Pile(item_type=Book, strict_type=True)
try:
    strict_library.include(Element())  # This should raise an error
except Exception as e:
    print(f"Error when trying to add non-Book to strict pile: {e}")

Error when trying to add non-Book to strict pile: Invalid item type in pile. Expected {<class '__main__.Book'>} or the subclasses


In [22]:
# Demonstrate how these methods affect the internal order
print("\nDemonstrating effect on internal order:")
ordered_library = Pile(
    items=[
        Book("Book A", "Author A"),
        Book("Book B", "Author B"),
        Book("Book C", "Author C"),
    ]
)
print(f"Initial order: {[book.title for book in ordered_library]}")

ordered_library.insert(1, Book("Book X", "Author X"))
print(f"Order after insert: {[book.title for book in ordered_library]}")

ordered_library.append(Book("Book Y", "Author Y"))
print(f"Order after append: {[book.title for book in ordered_library]}")

# Note on thread safety and asynchronous operations
print("\nNote: `Append` and `insert` write operations on Pile are thread-safe.")
print(
    "Note: Pile supports both sync and async operations. Use 'a'-prefixed methods for async (e.g., ainclude, aupdate)."
)


Demonstrating effect on internal order:
Initial order: ['Book A', 'Book B', 'Book C']
Order after insert: ['Book A', 'Book X', 'Book B', 'Book C']
Order after append: ['Book A', 'Book X', 'Book B', 'Book C', 'Book Y']

Note: `Append` and `insert` write operations on Pile are thread-safe.
Note: Pile supports both sync and async operations. Use 'a'-prefixed methods for async (e.g., ainclude, aupdate).


## Pop/Remove/Exclude/Clear

In [23]:
# Initialize a Pile with some books
library = Pile(
    [
        Book("1984", "George Orwell"),
        Book("To Kill a Mockingbird", "Harper Lee"),
        Book("Pride and Prejudice", "Jane Austen"),
        Book("The Great Gatsby", "F. Scott Fitzgerald"),
        Book("Moby-Dick", "Herman Melville"),
    ],
    item_type=Book,
)

print("Initial library:")
print(library)

# pop: Remove and return an item by index or key
print("\nDemonstrating 'pop' method:")
removed_book = library.pop(2)  # Remove the book at index 2
print(f"Removed book: {removed_book}")
print(f"Library after pop: {library}")

# Try to pop a non-existent index
try:
    library.pop(10)
except Exception as e:
    print(f"Error when trying to pop non-existent index: {e}")

# remove: Remove a specific item
print("\nDemonstrating 'remove' method:")
book_to_remove = library[0]
library.remove(book_to_remove)
print(f"Removed book: {book_to_remove}")
print(f"Library after remove: {library}")

# Try to remove a non-existent item
try:
    library.remove(Book("Non-existent Book", "Unknown Author"))
except Exception as e:
    print(f"Error when trying to remove non-existent book: {e}")

# exclude: Remove item(s) if present
print("\nDemonstrating 'exclude' method:")
books_to_exclude = [
    Book("The Great Gatsby", "F. Scott Fitzgerald"),
    Book("Non-existent Book", "Unknown Author"),
]
library.exclude(books_to_exclude)
print(f"Library after exclude: {library}")

# clear: Remove all items
print("\nDemonstrating 'clear' method:")
library.clear()
print(f"Library after clear: {library}")

# is_empty: Check if the pile is empty
print("\nDemonstrating 'is_empty' method:")
print(f"Is the library empty? {library.is_empty()}")

# Add a book to the empty library
library.include(Book("Dune", "Frank Herbert"))
print(f"Is the library empty after adding a book? {library.is_empty()}")

# Demonstrate the difference between remove and exclude
print("\nDemonstrating the difference between remove and exclude:")
library = Pile(
    [
        Book("Book A", "Author A"),
        Book("Book B", "Author B"),
        Book("Book C", "Author C"),
    ],
    item_type=Book,
)

print("Initial library:", library)

# remove raises an exception if the item is not found
try:
    library.remove(Book("Non-existent Book", "Unknown Author"))
except Exception as e:
    print(f"Error when using remove: {e}")

# exclude does not raise an exception if the item is not found
library.exclude(Book("Non-existent Book", "Unknown Author"))
print("Library after exclude (no exception raised):", library)

# Demonstrate how these methods affect the internal order
print("\nDemonstrating effect on internal order:")
ordered_library = Pile(
    [
        Book("Book A", "Author A"),
        Book("Book B", "Author B"),
        Book("Book C", "Author C"),
        Book("Book D", "Author D"),
        Book("Book E", "Author E"),
    ],
    item_type=Book,
)

print("Initial order:", [book.title for book in ordered_library])

ordered_library.pop(2)
print("Order after pop:", [book.title for book in ordered_library])

ordered_library.remove(ordered_library[0])
print("Order after remove:", [book.title for book in ordered_library])

ordered_library.exclude([ordered_library[0], Book("Non-existent", "Author")])
print("Order after exclude:", [book.title for book in ordered_library])

# Note on thread safety and asynchronous operations
print("\nNote: `pop`, `clear` operations on Pile are thread-safe.")
print(
    "Note: Pile supports both sync and async operations. Use 'a'-prefixed methods for async (e.g., apop, aremove)."
)

Initial library:
Pile(5)

Demonstrating 'pop' method:
Removed book: Book(ln_id=ln23e1.., timestamp=2024-10-01T13:29+00:00)
Library after pop: Pile(4)
Error when trying to pop non-existent index: Item not found. Error: index 10 item not found

Demonstrating 'remove' method:
Removed book: Book(ln_id=ln49c0.., timestamp=2024-10-01T13:29+00:00)
Library after remove: Pile(3)
Error when trying to remove non-existent book: Book(ln_id=ln754c.., timestamp=2024-10-01T13:29+00:00)

Demonstrating 'exclude' method:
Library after exclude: Pile(3)

Demonstrating 'clear' method:
Library after clear: Pile(0)

Demonstrating 'is_empty' method:
Is the library empty? True
Is the library empty after adding a book? False

Demonstrating the difference between remove and exclude:
Initial library: Pile(3)
Error when using remove: Book(ln_id=ln6526.., timestamp=2024-10-01T13:29+00:00)
Library after exclude (no exception raised): Pile(3)

Demonstrating effect on internal order:
Initial order: ['Book A', 'Book B',

## Get / Keys / Values / Items / Size / len / list

In [24]:
# Initialize a Pile with some books
library = Pile(
    [
        Book("1984", "George Orwell"),
        Book("To Kill a Mockingbird", "Harper Lee"),
        Book("Pride and Prejudice", "Jane Austen"),
        Book("The Great Gatsby", "F. Scott Fitzgerald"),
        Book("Moby-Dick", "Herman Melville"),
    ],
    item_type=Book,
)

print("Initial library:")
print(library)

# get: Retrieve an item by key with a default value
print("\nDemonstrating 'get' method:")
book = library.get(0, "Book not found")
print(f"Book at index 0: {book}")

non_existent_book = library.get(10, "Book not found")
print(f"Book at index 10: {non_existent_book}")

# keys: Get all keys (Lion IDs) in the Pile
print("\nDemonstrating 'keys' method:")
book_keys = library.keys()
print(f"Book keys: {book_keys}")

# values: Get all values in the Pile
print("\nDemonstrating 'values' method:")
books = library.values()
print(f"Books: {books}")

# items: Get all (key, value) pairs in the Pile
print("\nDemonstrating 'items' method:")
book_items = library.items()
print(f"Book items: {book_items}")

# size: Get the number of items in the Pile
print("\nDemonstrating 'size' method:")
library_size = library.size()
print(f"Library size: {library_size}")

# len: Get the number of items in the Pile using len()
print("\nDemonstrating 'len' function:")
library_len = len(library)
print(f"Library length: {library_len}")

# list: Convert the Pile to a list
print("\nDemonstrating conversion to list:")
book_list = list(library)
print(f"Book list: {book_list}")

# Demonstrate how these methods can be used in practical scenarios
print("\nPractical usage scenarios:")

# 1. Finding a book by its index
print("\n1. Finding a book by its index:")
third_book = library.get(2, "Book not found")
print(f"The third book in the library is: {third_book}")

# 2. Listing all book titles
print("\n2. Listing all book titles:")
book_titles = [book.title for book in library.values()]
print(f"Book titles: {book_titles}")

# 3. Creating a dictionary of books with their Lion IDs
print("\n3. Creating a dictionary of books with their Lion IDs:")
book_dict = {key: book.title for key, book in library.items()}
print(f"Book dictionary: {book_dict}")

# 4. Checking if the library has a specific number of books
print("\n4. Checking if the library has a specific number of books:")
expected_size = 5
if library.size() == expected_size:
    print(f"The library has the expected {expected_size} books.")
else:
    print(
        f"The library size ({library.size()}) doesn't match the expected size ({expected_size})."
    )

# 5. Processing books in batches
print("\n5. Processing books in batches:")
batch_size = 2
for i in range(0, len(library), batch_size):
    batch = list(library)[i : i + batch_size]
    print(f"Processing batch {i//batch_size + 1}: {batch}")

# Note on performance considerations
print("\nNote: For large collections, consider the performance implications:")
print("- 'keys()', 'values()', and 'items()' return sequences, not iterators.")
print("- For very large Piles, these methods may consume significant memory.")
print("- Consider using iteration directly on the Pile for large datasets.")

# Demonstrate iteration directly on the Pile
print("\nDemonstrating direct iteration on the Pile:")
for book in library:
    print(book.title)

Initial library:
Pile(5)

Demonstrating 'get' method:
Book at index 0: Book(ln_id=ln8ea1.., timestamp=2024-10-01T13:29+00:00)
Book at index 10: Book not found

Demonstrating 'keys' method:
Book keys: ['ln8ea157-810a903d0be486a2-dd-746d-cc399a8c3e425d', 'lned14db-60b1656-8ec066-d7228637fe730d-9b3b6e4a8', 'ln775dd30dac6ac67-9c1f-2e-bb-f1648a9c093350844e8', 'lne6a4836-cf7ff336798-e05fde66-12b0b-de172508630', 'ln7bd495e2-93-c8fe59d6bd11c3e0c0ac6-56a8-294cc01']

Demonstrating 'values' method:
Books: [Book('1984' by George Orwell), Book('To Kill a Mockingbird' by Harper Lee), Book('Pride and Prejudice' by Jane Austen), Book('The Great Gatsby' by F. Scott Fitzgerald), Book('Moby-Dick' by Herman Melville)]

Demonstrating 'items' method:
Book items: [('ln8ea157-810a903d0be486a2-dd-746d-cc399a8c3e425d', Book('1984' by George Orwell)), ('lned14db-60b1656-8ec066-d7228637fe730d-9b3b6e4a8', Book('To Kill a Mockingbird' by Harper Lee)), ('ln775dd30dac6ac67-9c1f-2e-bb-f1648a9c093350844e8', Book('Pride

## Async Methods

In [25]:
import asyncio


# Helper function to simulate an asynchronous operation
async def async_operation(duration):
    await asyncio.sleep(duration)


# Initialize a Pile with some books
library = Pile(
    [
        Book("1984", "George Orwell"),
        Book("To Kill a Mockingbird", "Harper Lee"),
        Book("Pride and Prejudice", "Jane Austen"),
    ],
    item_type=Book,
)

print("Initial library:")
print(library)

Initial library:
Pile(3)


In [26]:
# Sync
library.include(Book("The Great Gatsby", "F. Scott Fitzgerald"))
print("Sync include result:", library)

# Async
await library.ainclude(Book("Moby-Dick", "Herman Melville"))
print("Async include result:", library)

Sync include result: Pile(4)
Async include result: Pile(5)


**Explanation**: The `include` method adds an item to the Pile immediately. The ainclude method is the asynchronous version, which allows other operations to occur while the inclusion is being processed.

In [27]:
# Sync
popped_book = library.pop(0)
print("Sync pop result:", popped_book)

# Async
apopped_book = await library.apop(0)
print("Async pop result:", apopped_book)

Sync pop result: Book(ln_id=ln2c0e.., timestamp=2024-10-01T13:29+00:00)
Async pop result: Book(ln_id=lnf1b6.., timestamp=2024-10-01T13:29+00:00)


**Explanation**: The pop method removes and returns an item from the Pile immediately. The apop method is the asynchronous version, which can be useful when removing items in a concurrent environment.

In [28]:
book_to_remove = library[0]

# Sync
library.remove(book_to_remove)
print("Sync remove result:", library)

# Async
await library.aremove(library[0])
print("Async remove result:", library)

Sync remove result: Pile(2)
Async remove result: Pile(1)


**Explanation**: The remove method removes a specific item from the Pile immediately. The aremove method is the asynchronous version, allowing for non-blocking removal operations.

In [29]:
new_books = [Book("Dune", "Frank Herbert"), Book("Neuromancer", "William Gibson")]

# Sync
library.update(new_books)
print("Sync update result:", library)

# Async
await library.aupdate(
    [Book("Foundation", "Isaac Asimov"), Book("The Hobbit", "J.R.R. Tolkien")]
)
print("Async update result:", library)

Sync update result: Pile(3)
Async update result: Pile(5)


In [30]:
# Sync
sync_get = library.get(0, "Not found")
print("Sync get result:", sync_get)

# Async
async_get = await library.aget(0, "Not found")
print("Async get result:", async_get)

Sync get result: Book(ln_id=ln7fb6.., timestamp=2024-10-01T13:29+00:00)
Async get result: Book(ln_id=ln7fb6.., timestamp=2024-10-01T13:29+00:00)


In [31]:
# We'll use new piles for this example
sync_pile = Pile(library.values(), item_type=Book)
async_pile = Pile(library.values(), item_type=Book)

# Sync
sync_pile.clear()
print("Sync clear result:", sync_pile)

# Async
await async_pile.aclear()
print("Async clear result:", async_pile)

Sync clear result: Pile(0)
Async clear result: Pile(0)


In [32]:
library = Pile(
    [
        Book("Book A", "Author A"),
        Book("Book B", "Author B"),
        Book("Book C", "Author C"),
    ],
    item_type=Book,
)

print("Sync iteration:")
for book in library:
    print(book)

print("\nAsync iteration:")
async for book in library:
    print(book)
    await async_operation(0.1)  # Simulate some async processing

Sync iteration:
Book(ln_id=ln5094.., timestamp=2024-10-01T13:29+00:00)
Book(ln_id=ln20da.., timestamp=2024-10-01T13:29+00:00)
Book(ln_id=ln7c69.., timestamp=2024-10-01T13:29+00:00)

Async iteration:
Book(ln_id=ln5094.., timestamp=2024-10-01T13:29+00:00)
Book(ln_id=ln20da.., timestamp=2024-10-01T13:29+00:00)
Book(ln_id=ln7c69.., timestamp=2024-10-01T13:29+00:00)


## Others

In [33]:
# Helper function to create a pile of books
def create_book_pile(*titles_and_authors):
    return Pile(
        [Book(title, author) for title, author in titles_and_authors], item_type=Book
    )

In [34]:
# Or

pile1 = create_book_pile(
    ("1984", "George Orwell"), ("To Kill a Mockingbird", "Harper Lee")
)
pile2 = create_book_pile(
    ("Pride and Prejudice", "Jane Austen"), ("1984", "George Orwell")
)

union_pile = pile1 | pile2
print("Union result:", union_pile)

Union result: Pile(4)


In [35]:
# ior

pile1 |= pile2
print("In-place union result:", pile1)

In-place union result: Pile(4)


In [36]:
# xor

pile1 = create_book_pile(
    ("1984", "George Orwell"), ("To Kill a Mockingbird", "Harper Lee")
)
pile2 = create_book_pile(
    ("Pride and Prejudice", "Jane Austen"), ("1984", "George Orwell")
)

sym_diff_pile = pile1 ^ pile2
print("Symmetric difference result:", sym_diff_pile)

Symmetric difference result: Pile(4)


In [37]:
# ixor

pile1 ^= pile2
print("In-place symmetric difference result:", pile1)

In-place symmetric difference result: Pile(4)


In [38]:
# and

pile1 = create_book_pile(
    ("1984", "George Orwell"), ("To Kill a Mockingbird", "Harper Lee")
)
pile2 = create_book_pile(
    ("Pride and Prejudice", "Jane Austen"), ("1984", "George Orwell")
)

intersection_pile = pile1 & pile2
print("Intersection result:", intersection_pile)

Intersection result: Pile(0)


In [39]:
# iand

pile1 &= pile2
print("In-place intersection result:", pile1)

In-place intersection result: Pile(0)


In [40]:
# bool

empty_pile = Pile(item_type=Book)
non_empty_pile = create_book_pile(("1984", "George Orwell"))

print("Empty pile is truthy:", bool(empty_pile))
print("Non-empty pile is truthy:", bool(non_empty_pile))

Empty pile is truthy: False
Non-empty pile is truthy: True


In [41]:
book_pile = create_book_pile(
    ("1984", "George Orwell"), ("To Kill a Mockingbird", "Harper Lee")
)
print("Str representation:", str(book_pile))
print("Repr representation:", repr(book_pile))

Str representation: Pile(2)
Repr representation: Pile(2)


In [42]:
pile_dict = {
    "pile_": [
        {"title": "1984", "author": "George Orwell"},
        {"title": "To Kill a Mockingbird", "author": "Harper Lee"},
    ],
    "item_type": "Book",
}

new_pile = Pile.from_dict(pile_dict)
print("Pile created from dict:", new_pile)

Pile created from dict: Pile(0)


## Key Concepts

1. **Elements**: Base objects in lion_core. Inherit from `Element` class.
2. **Pile**: Thread-safe, ordered collection of Elements.
3. **Asynchronous Support**: Many operations have sync and async versions.

## Best Practices

1. Use type hints:
   ```python
   from lion_core.generic.pile import Pile
   def process(elements: Pile[YourElement]) -> None: ...
   ```

2. Choose sync or async consistently within a module.

3. Handle lionabc exceptions:
   ```python
   from lionabc.exceptions import LionValueError, LionTypeError
   try:
       # lion_core operation
   except (LionValueError, LionTypeError) as e:
       # Handle error
   ```

4. Don't modify `ln_id` of Elements.

5. Use Pile's methods over direct list operations.

6. In custom Elements, always call `super().__init__()`.

7. Use `to_dict()` and `from_dict()` for serialization.

## Performance Tips

1. For large collections, prefer direct iteration over `keys()`, `values()`, `items()`.
2. Use async methods for I/O bound or high-concurrency scenarios.
3. Use batch operations where available.

## Extending lion_core

1. Inherit from appropriate base classes when creating custom managers.
2. Write comprehensive tests for extensions.

## Common Pitfalls

1. Avoid mixing sync and async code.
2. Be cautious about modifying a Pile during iteration.
3. Don't ignore type check errors.
4. Be careful when overriding core class methods.

Remember to consult the official documentation for detailed information.